# Projet - EDA

In [2]:
import sys
sys.path.append('../src')
from utils import *

df = getDF('../data/Sports_and_Outdoors_5.json.gz')

# Garder seulement les 100 premières lignes
df = df.head(100)

In [3]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,vote,image
0,5.0,True,"06 3, 2015",A180LQZBUWVOLF,0000032034,Michelle A,What a spectacular tutu! Very slimming.,Five Stars,1433289600,NaN,NaN,NaN
1,1.0,True,"04 1, 2015",ATMFGKU5SVEYY,0000032034,Crystal R,What the heck? Is this a tutu for nuns? I know...,Is this a tutu for nuns?!,1427846400,NaN,NaN,NaN
2,5.0,True,"01 13, 2015",A1QE70QBJ8U6ZG,0000032034,darla Landreth,Exactly what we were looking for!,Five Stars,1421107200,NaN,NaN,NaN
3,5.0,True,"12 23, 2014",A22CP6Z73MZTYU,0000032034,L. Huynh,I used this skirt for a Halloween costume and ...,I liked that the elastic waist didn't dig in (...,1419292800,NaN,NaN,NaN
4,4.0,True,"12 15, 2014",A22L28G8NRNLLN,0000032034,McKenna,This is thick enough that you can't see throug...,This is thick enough that you can't see throug...,1418601600,NaN,NaN,NaN


In [10]:
from mlxtend.frequent_patterns import apriori, association_rules, fpmax
from sklearn.preprocessing import LabelEncoder

# Nettoyage des données
df_cleaned = df[['reviewerID', 'asin']]

# Encodage des données
label_encoder = LabelEncoder()
df_cleaned['reviewerID_encoded'] = label_encoder.fit_transform(df_cleaned['reviewerID'])
df_cleaned['asin_encoded'] = label_encoder.fit_transform(df_cleaned['asin'])

# Créer une colonne pour les occurrences
df_cleaned['occurrence'] = 1
print(df_cleaned.head())
# Pivoter la table pour avoir les occurrences par utilisateur et produit
pivoted_df = df_cleaned.pivot_table(index='reviewerID_encoded', columns='asin_encoded', values='occurrence', fill_value=0)

# Transformer les valeurs en binaire
binary_df = (pivoted_df > 0).astype(int) # Changer le seuil
binary_df.head()

       reviewerID        asin  reviewerID_encoded  asin_encoded  occurrence
0  A180LQZBUWVOLF  0000032034                   4             0           1
1   ATMFGKU5SVEYY  0000032034                  91             0           1
2  A1QE70QBJ8U6ZG  0000032034                  21             0           1
3  A22CP6Z73MZTYU  0000032034                  31             0           1
4  A22L28G8NRNLLN  0000032034                  32             0           1


/tmp/ipykernel_12530/1360007512.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['reviewerID_encoded'] = label_encoder.fit_transform(df_cleaned['reviewerID'])
/tmp/ipykernel_12530/1360007512.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['asin_encoded'] = label_encoder.fit_transform(df_cleaned['asin'])
/tmp/ipykernel_12530/1360007512.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

asin_encoded,0,1,2,3,4,5
reviewerID_encoded,,,,,,
0,0,1,0,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,1,0,0,0,0,0


In [5]:
# Extraction des motifs fréquents
frequent_itemsets = apriori(binary_df, min_support=0.01, use_colnames=True)

# Afficher les motifs fréquents
print("Motifs fréquents :")
print(frequent_itemsets)

Motifs fréquents :
    support itemsets
0  0.051546      (0)
1  0.226804      (1)
2  0.432990      (2)
3  0.134021      (3)
4  0.134021      (4)
5  0.030928      (5)
6  0.010309   (1, 2)


/home/elouan/epita/sciag/EDA/venv/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [6]:
# Extraction des motifs fermés
closed_itemsets = fpmax(binary_df, min_support=0.01, use_colnames=True)

# Afficher les motifs fermés
print("Motifs fermés :")
print(closed_itemsets)

Motifs fermés :
    support itemsets
0  0.030928      (5)
1  0.051546      (0)
2  0.134021      (3)
3  0.134021      (4)
4  0.010309   (1, 2)


/home/elouan/epita/sciag/EDA/venv/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [7]:
from pattern_mining import create_and_fit_lcm

# Extraction des motifs fréquents avec LCM
frequent_itemsets = create_and_fit_lcm(binary_df, supp_ratio=0.5)
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
frequent_itemsets

TypeError: 'int' object is not iterable

In [ ]:
# Extraction des règles d'association
association_rules_df = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Afficher les règles d'association
print("Règles d'association les plus confiantes :")
print(association_rules_df.sort_values(by='confidence', ascending=False))

In [ ]:
# Analyse des sous-populations
subpopulation_id = 'ID_DE_LA_SOUS_POPULATION' # Remplacer par l'ID de la sous-population qu'on souhaite analyser
subpopulation = df_cleaned[df_cleaned['reviewerID'] == subpopulation_id]
subpopulation_frequent_itemsets = apriori(subpopulation.drop(['reviewerID', 'asin'], axis=1), min_support=0.01, use_colnames=True)

# Afficher les motifs fréquents pour la sous-population
print("Motifs fréquents pour la sous-population spécifique :")
print(subpopulation_frequent_itemsets)

In [ ]:
# Motifs de compression des données
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
compressed_data = pca.fit_transform(df_cleaned.drop(['reviewerID', 'asin'], axis=1))

# Afficher les motifs de compression des données
print("Composantes principales de la compression des données :")
print(pca.components_)